In [ ]:
# !yes | add-apt-repository ppa:mscore-ubuntu/mscore3-stable &> /dev/null
# !apt update &> /dev/null
# !apt install musescore3 &> /dev/null
# print("MuseScore installation finished")


In [13]:
import music21
import random
import os

from music21 import *

import pandas as pd
from fractions import Fraction

music_score_path = 'C:/Program Files/MuseScore 4/bin/MuseScore4.exe'
music21.environment.set('musescoreDirectPNGPath', music_score_path)


In [2]:
!mkdir output


In [3]:
def round_if_more_than_2_decimal_places(value):
    value_str = f"{value:.10f}"  
    decimal_part = value_str.split(".")[1].rstrip("0")
    if len(decimal_part) > 3:
        return round(value, 2)
    return value

In [4]:
def Stream2PIG(sf, test=False, n=0, beam=0, time_unit=0.5):
    """
    Convert a Stream to PIG CSV format using Pandas.
    
    sf: music21 Stream object.
    fname: Output CSV file name.
    beam: 0 for right hand, 1 for left hand.
    time_unit: Time scaling factor.
    """
    data = []
    idx = 0
    name = None
    pre_onset = 0
    
    for measure in sf.getElementsByClass(stream.Measure):
        measure_idx = measure.measureNumber if measure.measureNumber is not None else idx
        
        for elem in measure.notesAndRests:
            onset = pre_onset
            value = elem.quarterLength

            if isinstance(elem.quarterLength, Fraction):
                if elem.quarterLength.denominator != 1:
                    value = float(elem.quarterLength)
                    
            offset = onset + value
            onset = round(onset, 2)
            offset = round(offset, 2)
            
            if isinstance(elem, note.Note):
                name = elem.nameWithOctave.replace('-', 'b')
                fingering = None
                if elem.articulations:
                    for art in elem.articulations:
                        if isinstance(art, articulations.Fingering):
                            fingering = art.fingerNumber
                
                
                if fingering is not None and (fingering < 1 or fingering > 5):
                    print(f"NOTE: {name}, Fingering {fingering} outside the valid range (1-5) at measure (idx): {measure_idx}")
                    measure.show()  
                if fingering is None:
                    print(f"NOTE: {name}, Fingering None outside the valid range (1-5) at measure (idx): {measure_idx}")
                    measure.show() 
        

                if beam == 0:
                    data.append([idx, round(onset, 3), round(offset, 3), name, 0, 0, beam, fingering if fingering is not None else 99])
                else:
                    data.append([idx, round(onset, 3), round(offset, 3), name, 0, 0, beam, -fingering if fingering is not None else 99])
                idx += 1
            
            elif isinstance(elem, chord.Chord):
                name_pitch = [pitch.nameWithOctave.replace('-', 'b') for pitch in elem.pitches]
                fingering = []
                
                for art in elem.articulations:
                    if isinstance(art, articulations.Fingering):
                        finger = art.fingerNumber if art.fingerNumber is not None else 99
                        fingering.append(finger)
                
                while len(fingering) < len(name_pitch):
                    fingering.append(99)
                
                for i in range(len(name_pitch)):
                    if int(fingering[i]) < 1 or int(fingering[i]) > 5:
                        print(f"NOTE: {name_pitch[i]} Fingering {fingering[i]} outside the valid range (1-5) at measure (idx): {measure_idx}.")
                        measure.show()  
                    if fingering is None:
                        print(f"NOTE: {name_pitch[i]} Fingering None outside the valid range (1-5) at measure (idx): {measure_idx}.")
                        measure.show()  
                    if beam == 0:
                        data.append([idx, round(onset, 3), round(offset, 3), name_pitch[i], 0, 0, beam, fingering[i] if fingering is not None else 99])
                    else:
                        data.append([idx, round(onset, 3), round(offset, 3), name_pitch[i], 0, 0, beam, -fingering[i] if fingering is not None else 99])    
                    idx += 1
            pre_onset = offset
    df = pd.DataFrame(data, columns=["idx", "onset", "offset", "name", "col1", "col2", "beam", "fingering"])
    
    return df


In [5]:
def xml_to_csv(input_file, output_file):
    
    piece = music21.converter.parse(input_file)
    right_hand = piece.parts[0]
    left_hand = piece.parts[1]
    
    df_right = Stream2PIG(right_hand, test=True, n=11, beam=0, time_unit=1.0)
    df_left = Stream2PIG(left_hand, test=True, n=11, beam=1, time_unit=1.0)
    header = ["ID", "onset_time", "offset_time", "pitch", \
          "onset_vel", "offset_vel", "channel", "finger"]
    result = pd.concat([df_right, df_left], axis=0, join='outer')
    
    result.to_csv(output_file, index=False, header=False)


In [15]:
for dirname, _, filenames in os.walk('../data/own_mxl'):
    for filename in filenames:
        print(filename)
        full_name = os.path.join(dirname, filename)
        xml_to_csv(full_name, f'output/{filename[:-4]}.csv')


1.mxl
10.mxl
11.mxl
12.mxl
13.mxl
14.mxl
15.mxl
16.mxl
17.mxl
18.mxl
19.mxl
2.mxl
20.mxl
21.mxl
22.mxl
23.mxl
24.mxl
25.mxl
26.mxl
27.mxl
28.mxl
29.mxl
3.mxl
30.mxl
31.mxl
32.mxl
33.mxl
34.mxl
35.mxl
36.mxl
37.mxl
38.mxl
39.mxl
4.mxl
40.mxl
41.mxl
42.mxl
45.mxl
46.mxl
47.mxl
48.mxl
49.mxl
5.mxl
50.mxl
50_Little_Etudes_No._11_-_20__Isaak_Berkovich.mxl
50_Little_Etudes_No._1_-_10__Isaak_Berkovich.mxl
51.mxl
52.mxl
53.mxl
54.mxl
57.mxl
58.mxl
59.mxl
6.mxl
61.mxl
62.mxl
63.mxl
64.mxl
66.mxl
67.mxl
69.mxl
7.mxl
70.mxl
72.mxl
74.mxl
75.mxl
76.mxl
77.mxl
78.mxl
79.mxl
8.mxl
80.mxl
9.mxl
Air_on_the_G_string_with_fingering.mxl
All of Me - Lead Sheet with Fingering.mxl
Bach G minor arr. Luo Ni (with fingering).mxl
Canon_in_D_By_OMID.mxl
Chopin_-_Nocturne_No.20_with_fingering_in_C_Minor_Op._posth.mxl


C:\python\Lib\site-packages\music21\musicxml\xmlToM21.py:5254: MusicXMLWarning: Could not import wedge: Error in getting DynamicWedges
  warnings.warn(f'Could not import {tag}: {excep}', MusicXMLWarning)


Cuphead - Inkwell Isle (Beginner-Intermediate with Fingering) – Kristofer Maddigan.mxl
Czerny - School Of Velocity - Op. 299 No. 11.mxl
Czerny - School Of Velocity - Op. 299 No. 2.mxl
Czerny - School Of Velocity - Op. 299 No. 3.mxl
Czerny - School Of Velocity - Op. 299 No. 4.mxl
Czerny - School Of Velocity - Op. 299 No. 5.mxl
Czerny - School Of Velocity - Op. 299 No. 6.mxl
Czerny - School Of Velocity - Op. 299 No. 8.mxl
Czerny - School Of Velocity - Op. 299 No. 9.mxl
Czerny_-_School_Of_Velocity_-_Op._299_No._1.mxl
Golden Hour - JVKE (with fingering).mxl
Great Fairy Fountain - Zelda (with fingering).mxl
Greensleeves_Piano_Debutant.mxl
Gymnopedie1.mxl
Invention_4_by_J_S_Bach_BWV_775_for_Piano_with_fingering.mxl
Mariage_dAmour_arr_Elpiano_w_fingering.mxl
Mazurka.mxl
Minuet _in_F_Major_Anna_Bachmxl.mxl
Minuet_in_G_Major_Bach.mxl
Nerevar Rising - Morrowind – Jeremy Soule Morrowind Theme with fingering.mxl
Nostalgie – Olivier Toussaint.mxl


C:\python\Lib\site-packages\music21\musicxml\xmlToM21.py:4163: MusicXMLWarning: Line <dashes> stop without start
  warnings.warn('Line <' + mxObj.tag + '> stop without start', MusicXMLWarning)


Petite_Fleur____Becket_EASE.mxl
Prelude_No._1_in_C_Major_BWV_846_with_finger_suggestions_-_Johann_Sebastian_Bach (1).mxl
Preparatory_Exercises.mxl
Root Beer Rag Theme - Easy to read (with fingering).mxl
Scherzo by Franz Joseph Haydn.mxl
The_Art_of_Fugue_Contrapunctus_1_BWV_1080__Johann_Sebastian_Bach_includes_fingering.mxl


C:\python\Lib\site-packages\music21\musicxml\xmlToM21.py:4163: MusicXMLWarning: Line <bracket> stop without start
  warnings.warn('Line <' + mxObj.tag + '> stop without start', MusicXMLWarning)


The_Harebell_by_William_Smallwood3.mxl
Waltz_in_A_minor_2024_With_Fingerings.mxl
